# FITS Source Selection - Functions
## Preamble
Modules:

In [ ]:
using FITSIO
using DataFrames
using Plots
using StatPlots

using Base.Test # For testing functions
using Feather   # Saving data temporarily for tests

In [2]:
test_flag = true;

) in module Base at nullable.jl:238 overwritten in module NullableArrays at C:\Users\Robert\.julia\v0.6\NullableArrays\src\operators.jl:99.


## FITSIO Data Extraction
### Reading coordinates

In [3]:
"""
    FITS_Coords(fits_path)

Reads fits file at `fits_path` using the `FITSIO` module, pulls out the second table
and reads the `X` and `Y` variables into a `DataFrame`

Removes rows with values of `-1`, assumed to be nulled
"""
function FITS_Coords(fits_path)
    fits_file = FITS(fits_path)
    fits_coords = DataFrame(X = read(fits_file[2], "X"), Y = read(fits_file[2], "Y"))
    deleterows!(fits_coords, find(fits_coords[:, :X] .== -1)) # Remove nulled points
    
    close(fits_file)
    
    return fits_coords
end

FITS_Coords

In [4]:
if test_flag
let
    local fits_path = "D:\\Ubuntu-MPhys-Shared\\data_ftp\\nustar\\data\\obs\\00\\1\\10012001002\\event_cl\\nu10012001002A01_cl.evt.gz"
    #FITS_Coord_test_coords = FITS_Coords(fits_path)
    #Feather.write(".\\TestFiles\\FITS_Coords_test_file.feather", FITS_Coord_test_coords)# Saving test data
    local FITS_Coord_test_coords_prv = Feather.read(".\\TestFiles\\FITS_Coords_test_file.feather", nullable=false)
    local FITS_Coord_test_coords_cur = FITS_Coords(fits_path)

    @test FITS_Coord_test_coords_cur == FITS_Coord_test_coords_prv
end
end

Test Passed

### Reading pixel scale data

In [ ]:
"""
    FITS_pixel_scale(fits_path)

Reads fits file at `fits_path` using the `FITSIO` module, pulls out the second table
and reads the key values addociated with the `X` and `Y` pixel references, degrees and scale

Can continue to show coordinates along with selected columns by:

    `FITS_pixel_scale(fits_path)[[:coord, :ref_pix]]`

Keys are assumed to be from NuSTAR and of the value:

    "TCRPX38"   -   X ref pix
    "TCRVL38"   -   X ref deg
    "TCDLT38"   -   X ref scale

    "TCRPX39"   -   Y ref pix
    "TCRVL39"   -   Y ref deg
    "TCDLT39"   -   Y ref scale
"""
function FITS_pixel_scale(fits_path)
    fits_file = FITS(fits_path)
    fits_evt_table = fits_file[2]
    out_scale = DataFrame(coord=["X", "Y"])
    
    read_header(fits_evt_table) # Shows full header for the events table
    # Looking through, found that the reference pixel, coordinates and scale are given by:
    #read_key(fits_file, "key_name" OR position) # Shows key name, value and description

    x_ref_pix = read_key(fits_evt_table, "TCRPX38")[1] # X ref pix
    x_ref_deg = read_key(fits_evt_table, "TCRVL38")[1] # X ref deg
    x_ref_scl = read_key(fits_evt_table, "TCDLT38")[1] # X ref scale

    y_ref_pix = read_key(fits_evt_table, "TCRPX39")[1] # Y ref pix
    y_ref_deg = read_key(fits_evt_table, "TCRVL39")[1] # Y ref deg
    y_ref_scl = read_key(fits_evt_table, "TCDLT39")[1] # Y ref scale
    
    out_scale[:ref_pix] = [x_ref_pix, y_ref_pix]
    out_scale[:ref_deg] = [x_ref_deg, y_ref_deg]
    out_scale[:ref_scl] = [x_ref_scl, y_ref_scl]
    return out_scale
end

FITS_pixel_scale

In [6]:
if test_flag
let
    local fits_path = "D:\\Ubuntu-MPhys-Shared\\data_ftp\\nustar\\data\\obs\\00\\1\\10012001002\\event_cl\\nu10012001002A01_cl.evt.gz"
    #Feather.write(".\\TestFiles\\FITS_Pixel_Scale_test_file.feather", FITS_pixel_scale(fits_path))
    local FITS_Pixel_Scale_test_scale_prv = Feather.read(".\\TestFiles\\FITS_Pixel_Scale_test_file.feather", nullable=false)
    local FITS_Pixel_Scale_test_scale_cur = FITS_pixel_scale(fits_path)

    # Test only checks floating values, as strings get converted to `WeakRefString` type for speed
    @test FITS_Pixel_Scale_test_scale_prv[[:ref_pix, :ref_deg, :ref_scl]] == FITS_Pixel_Scale_test_scale_cur[[:ref_pix, :ref_deg, :ref_scl]]
end
end

Test Passed

## Stats
### Savitzky-Golay filter

In [ ]:
function sgolay(order, frameLen)
    S = (-(frameLen-1)/2:((frameLen-1)/2)) .^ (0:order)'
    (Q, R) = qr(S)
    B = Q*Q'
    G = Q / R'

    return B, G
end

sgolay (generic function with 1 method)

In [8]:
function sgolayfilt(x, order, frameLen)
    B = sgolay(order, frameLen)[1]
    x = x[:]

    @assert ndims(x) == 1

    ybegin = B[end:-1:round(Int, (frameLen-1)/2 + 2), :] * x[frameLen:-1:1, :]
    ycentre = filt(B[round(Int, (frameLen-1)./2 + 1), :], 1, x)
    yend = B[round(Int, (frameLen-1)/2):-1:1, :] * x[end:-1:end-(frameLen-1), :]

    return y = [ybegin; ycentre[frameLen:end, :]; yend]
end

sgolayfilt (generic function with 1 method)

## Source Detection

In [ ]:
function find_source_fwxm(coord_in; prcnt=0.5, filt_flag=true, verbose=true)
    data_out = similar(coord_in, 0)
    bnds_out = DataFrame(item = ["min_ind", "max_ind", "min_val", "max_val"])

    for coord in names(coord_in)
        hist_y = StatsBase.fit(Histogram, coord_in[coord], nbins=256, closed=:right).weights
        hist_x = StatsBase.fit(Histogram, coord_in[coord], nbins=256, closed=:right).edges[1]

        if filt_flag
            hist_y_old = copy(hist_y)
            hist_y = sgolayfilt(hist_y, 2, 21)
        end

        max_y_val, max_y_ind  = findmax(hist_y)

        max_ind = Int(max_y_ind + findfirst(hist_y[max_y_ind:end] .<= max_y_val*prcnt))
        min_ind = Int(max_y_ind - findfirst(hist_y[max_y_ind:-1:1] .<= max_y_val*prcnt))
        
        if verbose
        println("Selected region for $(coord): $(hist_x[min_ind]) ($(@sprintf("%.3f", hist_y[min_ind]))) ", 
                                           "to $(hist_x[max_ind]) ($(@sprintf("%.3f", hist_y[max_ind])))")
        end

        data_out = coord_in[Int(hist_x[min_ind]) .< coord_in[coord] .< Int(hist_x[max_ind]), :]
        bnds_out[coord] = [hist_x[min_ind], hist_x[max_ind], hist_y[min_ind], hist_y[max_ind]]
    end

    return data_out, bnds_out
end

find_source_fwxm (generic function with 1 method)

In [15]:
if test_flag
let
    local fits_path = "D:\\Ubuntu-MPhys-Shared\\data_ftp\\nustar\\data\\obs\\00\\1\\10012001002\\event_cl\\nu10012001002A01_cl.evt.gz"
    local fits_coord = FITS_Coords(fits_path);
    #Data_prv, Sel_prv = find_source_fwxm(fits_coord; prcnt=0.5, filt_flag=true)
    #Feather.write(".\\TestFiles\\FITS_Sel_Coord_Data_test_prv.feather", Data_prv)
    #Feather.write(".\\TestFiles\\FITS_Sel_Coord_Sel_test_prv.feather", Sel_prv)
    local FITS_Sel_Coord_Data_test_prv = Feather.read(".\\TestFiles\\FITS_Sel_Coord_Data_test_prv.feather", nullable=false)
    local FITS_Sel_Coord_Sel_test_prv = Feather.read(".\\TestFiles\\FITS_Sel_Coord_Sel_test_prv.feather", nullable=false)
    local FITS_Sel_Coord_Data_test_cur = find_source_fwxm(fits_coord; prcnt=0.5, filt_flag=true, verbose=false)[1]
    local FITS_Sel_Coord_Sel_test_cur = find_source_fwxm(fits_coord; prcnt=0.5, filt_flag=true, verbose=false)[2]

    @test FITS_Sel_Coord_Data_test_prv == FITS_Sel_Coord_Data_test_cur
    @test FITS_Sel_Coord_Sel_test_prv[[:X, :Y]] == FITS_Sel_Coord_Sel_test_cur[[:X, :Y]]
end
end

Test Passed

# Plotting